# Trim 3D TIFF File with Complete Tag Preservation
This notebook provides functionality to trim a 3-dimensional TIFF file while preserving all metadata, headers, and TIFF tags, with careful handling of extratags format, and preserving the software tag, which scanimage uses for saving a number of parameters.

In [5]:
# Import required libraries
import tifffile
from pathlib import Path

from tifftrim import trim_3d_tiff

/Users/fpbattaglia/src/TIFFTrim/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# (Previously this notebook re-defined trim_3d_tiff here.)
# Now we use the implementation from the installed/editable `tifftrim` package.

In [ ]:
# Example usage
input_file = "/Users/fpbattaglia/Dropbox/Data/477116_20251118_00001.tif"
output_file = "/Users/fpbattaglia/Dropbox/Data/477116_20251118_00001_0_6000.tif"

# Trim to keep frames 0 through 5999 (end exclusive)
trim_3d_tiff(input_file, output_file, 0, 6000)

# Verify the results
verify = False

